In [1]:
import requests
from bs4 import BeautifulSoup

#### 함수화 - 데이터 크롤링

In [2]:
def naver_webtoon(genre):
    
    dic = {}
    dic['unique_id'] = []
    dic['title'] = []
    dic['author'] = []
    dic['total_score'] = []
    dic['main_link'] = []
    dic['image_link'] = []
    dic['update_date'] = []
    
    # first url for title
    # get title with full name
    url = "https://comic.naver.com/webtoon/genre.nhn?view=list&order=ViewCount&genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#content > div.list_area.table_list_area > table > tbody > tr")
    for keyword in keywords:
        dic['title'].append(keyword.select_one("td > a > strong").text)
    
    
    # second url for the others
    url =  "https://comic.naver.com/webtoon/genre.nhn?genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".list_area > .img_list > li")


    # author, total score
    for keyword in keywords:
        dic['author'].append(keyword.select_one("dl > dd > a").text)
        dic['total_score'].append(keyword.select_one("dl > dd > .rating_type > strong").text)
        dic['update_date'].append(keyword.select_one("dl > dd.date2").text)

    # unique id 
    unique_id_list = dom.select("dl > dt > a")
    for i in range(len(unique_id_list)):
        idx = unique_id_list[i].get('href').index("=")
        dic['unique_id'].append(unique_id_list[i].get('href')[idx+1:])

    # main link
    main_link_list = dom.select("dl > dt > a")
    for i in range(len(main_link_list)):
        dic['main_link'].append("https://comic.naver.com" + main_link_list[i].get('href'))

    # image link
    image_link_list = dom.select(".list_area > .img_list > li > .thumb > a > img:nth-of-type(1)")
    for i in range(len(image_link_list)):
        dic['image_link'].append(image_link_list[i].get('src'))
        
    df = pd.DataFrame(dic)
    df[genre] = genre
    
    return df

#### 함수화 - 장르통합 "/"

In [24]:
def make_genre(df):
    genres = ["episode","omnibus","story","daily","comic","fantasy","action","drama","pure","sensibility","thrill","historical","sports"]
    df.fillna("", inplace=True)

    result = []
    for i in range(len(df[genres])):
        ls = list(filter(lambda value: value != "", df[genres].loc[i].values))
        result.append(ls)

    df['genre'] = np.array(["/".join(ls) for ls in result])
    df.drop(columns=genres, axis=1, inplace=True)
    return df